In [9]:
import altair as alt
from vega_datasets import data

source = alt.topo_feature(data.world_110m.url, 'countries')

base = alt.Chart(source).mark_geoshape(
    fill='#666666',
    stroke='white'
).properties(
    width=800,
    height=500
)

charts = base.project('equirectangular').properties(title='equirectangular')

charts

alt.Chart(...)

In [29]:
import altair as alt
from vega_datasets import data

# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Layering and configuring the components
alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='lightblue'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.3, opacity=0.8),
    alt.Chart(source).mark_geoshape(fill='yellow', stroke='black', opacity=0.6)
).project(
    'naturalEarth1'
).properties(width=750, height=500).configure_view(stroke=None)

alt.LayerChart(...)

In [30]:
import requests
import json

import pandas as pd

import altair as alt
from vega_datasets import data
from altair import datum

df = pd.read_csv('https://raw.githubusercontent.com/MariaKokshaikina/any-data/main/covid19_global_confirmed_cases%20(1).csv')

country_info = requests.get(
    'https://raw.githubusercontent.com/MariaKokshaikina/any-data/main/country_info.json'
).json()

df = df[df['country_region'].isin(country_info)]
df = df.sort_values('date', ascending=True)
df = df.tail(5000)

def timestamp(t):
    return pd.to_datetime(t).timestamp() * 1000

df['id'] = df['country_region'].map(lambda x: country_info[x]['numericCode'])
df['rate'] = df['confirmed_cases'] / df['country_region'].map(lambda x: country_info[x]['population'])
df['timestamp'] = df['date'].map(timestamp)

countries = alt.topo_feature(data.world_110m.url, 'countries')

slider = alt.binding_range(
    step=24 * 60 * 60 * 1000,
    min=df['timestamp'].min(), 
    max=df['timestamp'].max()
)

select_date = alt.selection_single(
    name="slider", 
    fields=['timestamp'],
    bind=slider, 
)

alt.Chart(countries).mark_geoshape()\
    .encode(color='rate:Q')\
    .add_selection(select_date)\
    .transform_filter(select_date)\
    .transform_lookup(
        lookup='id',
        from_=alt.LookupData(df, key='id', fields=['rate'])
    )\
    .project('equirectangular')\
    .properties(
        width=500,
        height=300,
        title='Title'
    )


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)>